# 外皮情報LV2からLV3へのコンバート

## 1. 外皮性能計算プログラム＋簡単な追加情報

- 外皮性能計算プログラムの入力情報に、用途別床面積を追加したものより、推定を行う。
- 現状の外皮性能計算プログラムで入力されるのは用途別に分かれていない外皮性能であるため、室用途別の性能を求めるためには、外皮を用途別に割り振る必要がある。
- 土間床等面積が0㎡より大きい場合、基礎断熱部は床下空間に属するものとみなし、室と床下空間の間の床の合計面積は土間床等面積の合計に等しいとみなす。室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）
- 住戸全体の外皮（床下空間に属するものを除く）、および、室と床下空間の間の床は、室用途別の床面積に応じて各用途に割り当てる。集合住宅の戸境壁等も同様の扱いとする。外皮性能は用途によらず均一に割り付ける。開口部は、外皮性能計算プログラムの入力情報より推定を開始する場合は、庇等の日射遮蔽部材を考慮した計算を行うため、方位、遮蔽部材やその寸法等が異なる場合は、別部材として扱う。
- 各部位の熱橋は、対応する一般部のU値の低減に置き換える。一般部の断熱材の厚を調整する。
- その他は、用途別情報が得られている場合と同様に求める。

### 《土間床等面積が0㎡より大きい場合の扱い》
$$ \qquad
A_{b}=min(A_主 + A_他 + A_非, \sum_{i}^{}S_{df,i})
\qquad (1)
\\
$$

- 入力値：
  - 室用途不明の空間に属する土間床等の部位iの面積$[m ^ 2]$：$S_{df,i}$

- 出力値：

  - 室と床下空間の間の床面積の合計$[m ^ 2]$：$A_{b}$

### 《室用途別の床面積に応じた用途別面積の推定》
$$ \qquad
A_{主,i,j}= \frac{A_{i,j}・A_主}{A_主 + A_他 + A_非}, \quad
A_{他,i,j}= \frac{A_{i,j}・A_他}{A_主 + A_他 + A_非}\quad, \quad
A_{非,i,j}= \frac{A_{i,j}・A_非}{A_主 + A_他 + A_非}\quad
\qquad (2),\;(3),\;(4)
\\
$$
$$ \qquad
L_{主,i,j}= \frac{L_{i,j}・A_主}{A_主 + A_他 + A_非}, \quad
L_{他,i,j}= \frac{L_{i,j}・A_他}{A_主 + A_他 + A_非}\quad, \quad
L_{非,i,j}= \frac{L_{i,j}・A_非}{A_主 + A_他 + A_非}\quad
\qquad (5),\;(6),\;(7)
\\
$$
- 入力値：

  - 主居室、その他の居室、非居室の床面積$[m ^ 2]$：$A_主$、$A_他$、$A_非$
  - 外皮の方位j、部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{i,j}$
  - 熱橋及び土間床等の方位jの外周部𝑖の長さ$[m]$：$L_{i,j}$

- 出力値：

  - 主居室、その他の居室、非居室の外皮の方位j、部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{主,i,j}$、$A_{他,i,j}$、$A_{非,i,j}$
  - 主居室、その他の居室、非居室の熱橋及び土間床等の方位jの外周部𝑖の面積$[m ^ 2]$：$L_{主,i,j}$、$L_{他,i,j}$、$L_{非,i,j}$
    

#### Function

In [2]:
def GetAreaByRoomUse( target_area, main_floor_area, other_floor_area, total_floor_area, space_type ):
    # target_area as double      : 方位j、部位iの面積または長さ[m]
    # total_floor_area as double : 床面積の合計[m]
    # main_floor_area as double  : 主たる居室の床面積[m]
    # other_floor_area as double : その他の居室の床面積[m]
    # space_type as string       : 空間種類　'main' or 'other' or 'nonliving' or 'underfloor'

    if space_type == 'main':
        rate = main_floor_area / total_floor_area
    elif space_type == 'other':
        rate = other_floor_area / total_floor_area
    elif space_type == 'nonliving':
        rate = ( total_floor_area - ( main_floor_area + other_floor_area ) ) / total_floor_area
    else :
        raise ValueError('指定された空間種類は定義されていません')  
    
    return target_area * rate

#### Example

In [3]:
area = GetAreaByRoomUse(50.0, 30.0, 60.0, 120.0, 'main')
area

12.5

## 2. コンバート
### 1) 共通

- 地域の区分、主たる居室の床面積(m<sup>2</sup>)、その他の床面積(m<sup>2</sup>)、床面積の合計(m<sup>2</sup>)、簡易外皮入力フラグの情報は維持する。

#### Input

```
{
    'Region'                      : # 地域の区分 1 ~ 8 の整数
    'IsSimplifiedInput'           : # bool値 簡易入力から生成したデータか否かを表す, LV1からの場合は常にTrueとなる。
    'MainOccupantRoomFloorArea'   : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea'  : # その他の居室の床面積(m2)
    'TotalFloorArea'              : # 床面積の合計(m2)
}
```

#### Output

```
{
    'Region'                      : # 地域の区分 1 ~ 8 の整数
    'IsSimplifiedInput'           : # bool値 簡易入力から生成したデータか否かを表す, LV1からの場合は常にTrueとなる。
    'MainOccupantRoomFloorArea'   : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea'  : # その他の居室の床面積(m2)
    'TotalFloorArea'              : # 床面積の合計(m2)
}
```

#### Function

In [4]:
def convert_common(d):
    return {
        'Region'                     : d['Region'],
        'IsSimplifiedInput'          : d['IsSimplifiedInput'],
        'MainOccupantRoomFloorArea'  : d['MainOccupantRoomFloorArea'],
        'OtherOccupantRoomFloorArea' : d['OtherOccupantRoomFloorArea'],
        'TotalFloorArea'             : d['TotalFloorArea']
    }

#### Example

In [5]:
convert_common({
    'Region' : 6,
    'IsSimplifiedInput': True,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0
})

{'IsSimplifiedInput': True,
 'MainOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloorArea': 60.0,
 'Region': 6,
 'TotalFloorArea': 120.0}

## 2-2. コンバート【分類：壁体構成】

- 地域の区分、壁体名称、一般部位構造種別、壁体種別、層構成（部材名称、厚さ[m]、熱伝導率[W/mK)、容積比熱[(J/(m<sup>3</sup>・K)]、平均熱貫流率[W/(m<sup>2</sup>・K)]、一般部熱貫流率[W/(m<sup>2</sup>・K)]、木造熱橋部計算方法、熱橋部面積比率[-]、熱橋部熱貫流率(W/[m<sup>2</sup>・K)]、熱橋長さ[m]、線熱貫流率[W/(m・K)]、補正熱貫流率[W/(m<sup>2</sup>・K)]の情報は維持する。

### Input

```
data = {
    'name'                        : # 壁体名称
    'structure'                   : # 一般部位構造種別 1 ~ 4 の整数
    'type'                        : # 壁体種別 1 ~ 6 の整数
    'Layers'                      : {
        'matrial'                 : # 部材名称
        'd'                       : # 厚さ(m)
        'lambda'                  : # 熱伝導率(W/(m・K))
        'Crho'                    : # 容積比熱(J/(m3・K))
    }
    'Ua'                          : # 平均熱貫流率(W/(m2・K))
    'Ug'                          : # 一般部熱貫流率(W/(m2・K))
    'HeatBridge'                  : # 木造熱橋部計算方法 1 ~ 3 の整数   一般部位構造種別=1
    'rHeatBridge'                 : # 熱橋部面積比率(-)               木造熱橋部計算方法=1 or 木造熱橋部計算方法=2
    'UHeatBridge'                 : # 熱橋部熱貫流率(W/(m2・K))       木造熱橋部計算方法=1 or 木造熱橋部計算方法=2
    'lHeatBridge'                 : # 熱橋長さ(m)                    一般部位構造種別=2 or 一般部位構造種別=3
    'psiHeatBridge'               : # 線熱貫流率(W/(m・K))            一般部位構造種別=2 or 一般部位構造種別=3
    'Ur'                          : # 補正熱貫流率(W/(m2・K))         木造熱橋部計算方法=3 or 一般部位構造種別=3
}
```

### Output

```
data = {
    'name'                        : # 壁体名称
    'structure'                   : # 一般部位構造種別 1 ~ 4 の整数
    'type'                        : # 壁体種別 1 ~ 6 の整数
    'Layers'                      : {
        'matrial'                 : # 部材名称
        'd'                       : # 厚さ(m)
        'lambda'                  : # 熱伝導率(W/(m・K))
        'Crho'                    : # 容積比熱(J/(m3・K))
    }
    'Ua'                          : # 平均熱貫流率(W/(m2・K))
    'Ug'                          : # 一般部熱貫流率(W/(m2・K))
    'HeatBridge'                  : # 木造熱橋部計算方法 1 ~ 3 の整数   一般部位構造種別=1
    'rHeatBridge'                 : # 熱橋部面積比率(-)               木造熱橋部計算方法=1 or 木造熱橋部計算方法=2
    'UHeatBridge'                 : # 熱橋部熱貫流率(W/(m2・K))       木造熱橋部計算方法=1 or 木造熱橋部計算方法=2
    'lHeatBridge'                 : # 熱橋長さ(m)                    一般部位構造種別=2 or 一般部位構造種別=3
    'psiHeatBridge'               : # 線熱貫流率(W/(m・K))            一般部位構造種別=2 or 一般部位構造種別=3
    'Ur'                          : # 補正熱貫流率(W/(m2・K))         木造熱橋部計算方法=3 or 一般部位構造種別=3
}
```

### Function

In [3]:
def convert_wall(d):
    return {
        'name'                        : d['name'],
        'structure'                   : d['structure'],
        'type'                        : d['type'],
        'Layers'                      : d['Layers'],
        'Ua'                          : d['Ua'],
        'Ug'                          : d['Ug'],
        'HeatBridge'                  : d['HeatBridge'],
        'rHeatBridge'                 : d['rHeatBridge'],
        'UHeatBridge'                 : d['UHeatBridge'],
        'lHeatBridge'                 : d['lHeatBridge'],
        'psiHeatBridge'               : d['psiHeatBridge'],
        'Ur'                          : d['Ur']
    }

### Example

In [4]:
convert_wall({
    'name'                        : 'Sample Wall',
    'structure'                   : 1,
    'type'                        : 2,
    'Layers'                      : [{'material': 'GW16K','d': 0.076,'rambda': 0.045,'Crho': 13},
                                     {'material': 'GPB','d': 0.0095,'rambda': 0.22,'Crho': 830}],
    'Ua'                          : 0.53,
    'Ug'                          : None,
    'HeatBridge'                  : None,
    'rHeatBridge'                 : None,
    'UHeatBridge'                 : None,
    'lHeatBridge'                 : None,
    'psiHeatBridge'               : None,
    'Ur'                          : None
})

{'HeatBridge': None,
 'Layers': [{'Crho': 13, 'd': 0.076, 'material': 'GW16K', 'rambda': 0.045},
  {'Crho': 830, 'd': 0.0095, 'material': 'GPB', 'rambda': 0.22}],
 'UHeatBridge': None,
 'Ua': 0.53,
 'Ug': None,
 'Ur': None,
 'lHeatBridge': None,
 'name': 'Sample Wall',
 'psiHeatBridge': None,
 'rHeatBridge': None,
 'structure': 1,
 'type': 2}

## 2-3. コンバート【分類：開口部】

- 開口部名称、日射熱取得率[-]、熱貫流率[W/(m<sup>2</sup>・K)]の情報は維持する。

### Input

```
data = {
    'name'                        : # 開口部名称
    'eta'                         : # 日射熱取得率(-)         簡易外皮入力フラグ=2
    'etaCooling'                  : # 日射熱取得率（夏期）(-)   簡易外皮入力フラグ=1
    'etaHeating'                  : # 日射熱取得率（冬期）(-)   簡易外皮入力フラグ=1
    'U'                           : # 熱貫流率(W/(m2・K))
}
```

### Output

```
data = {
    'name'                        : # 開口部名称
    'eta'                         : # 日射熱取得率(-)         簡易外皮入力フラグ=2
    'etaCooling'                  : # 日射熱取得率（夏期）(-)   簡易外皮入力フラグ=1
    'etaHeating'                  : # 日射熱取得率（冬期）(-)   簡易外皮入力フラグ=1
    'U'                           : # 熱貫流率(W/(m2・K))
}
```

### Function

In [5]:
def convert_window(d):
    return {
        'name'                        : d['name'],
        'eta'                         : d['eta'],
        'etaCooling'                  : d['etaCooling'],
        'etaHeating'                  : d['etaHeating'],
        'U'                           : d['U']
    }

### Example

In [6]:
convert_window({
    'name'                        : 'Sample Window',
    'eta'                         : 0.738,
    'etaCooling'                  : None,
    'etaHeating'                  : None,
    'U'                           : 4.65
})

{'U': 4.65,
 'eta': 0.738,
 'etaCooling': None,
 'etaHeating': None,
 'name': 'Sample Window'}

## 2-4. コンバート【分類：外部日除け】

- 外部日除け名称、窓上端と外部日除けの距離[m]、開口部高さ[m]、出巾[m]、取得日射熱補正係数[-]の情報は維持する。

### Input

```
data = {
    'name'                        : # 外部日除け名称
    'y1'                          : # 窓上端と外部日除けの距離(m)    簡易外皮入力フラグ=2
    'y2'                          : # 開口部高さ(m)                簡易外皮入力フラグ=2
    'z'                           : # 出巾(m)                     簡易外皮入力フラグ=2
    'fCooling'                    : # 取得日射熱補正係数（夏期）(-)   簡易外皮入力フラグ=1
    'fHeating'                    : # 取得日射熱補正係数（冬期）(-)   簡易外皮入力フラグ=1
}
```

### Output

```
data = {
    'name'                        : # 外部日除け名称
    'y1'                          : # 窓上端と外部日除けの距離(m)    簡易外皮入力フラグ=2
    'y2'                          : # 開口部高さ(m)                簡易外皮入力フラグ=2
    'z'                           : # 出巾(m)                     簡易外皮入力フラグ=2
    'fCooling'                    : # 取得日射熱補正係数（夏期）(-)   簡易外皮入力フラグ=1
    'fHeating'                    : # 取得日射熱補正係数（冬期）(-)   簡易外皮入力フラグ=1
}
```

### Function

In [7]:
def convert_sunshade(d):
    return {
        'name'                        : d['name'],
        'y1'                          : d['y1'],
        'y2'                          : d['y2'], 
        'z'                           : d['z'],
        'fCooling'                    : d['fCooling'],
        'fHeating'                    : d['fHeating']
    }

### Example

In [8]:
convert_sunshade({
    'name'                        : 'Sample Sunshade',
    'y1'                          : 1,
    'y2'                          : 1, 
    'z'                           : 1,
    'fCooling'                    : None,
    'fHeating'                    : None
})

{'fCooling': None,
 'fHeating': None,
 'name': 'Sample Sunshade',
 'y1': 1,
 'y2': 1,
 'z': 1}

## 2-5. コンバート【分類：部位情報】

- 壁体・開口部名称、外皮フラグ、日射フラグ、日射フラグ、非定常フラグ、外部日除け名称の情報は維持する。
- 新たに変数「空間種類」「隣接空間種類」を作成する。
- 用途別の部位面積[m<sup>2</sup>]、部位長さ[m<sup>2</sup>]を、室用途別の床面積より推定する。
- 土間床等面積が0㎡より大きい場合、基礎断熱部は床下空間に属するものとみなし、室と床下空間の間の床の合計面積は土間床等面積の合計に等しいとみなす。室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）

### Input

```
data = {
    'name'                        : # 壁体・開口部名称
    'FlagPerimeter'               : # 外皮フラグ 1 ~ 2 の整数
    'FlagSolarRadiation'          : # 日射フラグ 1 ~ 2 の整数
    'direction'                   : # 方位 1 ~ 14 の整数
    'FlagUnsteady'                : # 非定常フラグ 1 ~ 2 の整数
    'A'                           : # 部位面積(m2)
    'L'                           : # 部位長さ(m)
    'sunshade'                    : # 外部日除け名称
}
```

### Output

```
data = {
    'name'                        : # 壁体・開口部名称
    'Room'                        : # 空間種類
    'FlagPerimeter'               : # 外皮フラグ 1 ~ 2 の整数
    'FlagSolarRadiation'          : # 日射フラグ 1 ~ 2 の整数
    'direction'                   : # 方位 1 ~ 14 の整数
    'AdjacentSpace'               : # 隣接空間種類
    'FlagUnsteady'                : # 非定常フラグ 1 ~ 2 の整数
    'A'                           : # 部位面積(m2)
    'L'                           : # 部位長さ(m)
    'sunshade'                    : # 外部日除け名称
}
```

### Function

In [9]:
def convert_element(d_common,d_elements):

    for d_element in d_elements:
        d_element['AdjacentSpace'] = None
    AEarthFloor=0
    for d in d_elements:
        if d['direction']==15:
            AEarthFloor=d['A']        
    if AEarthFloor>0:
        d_elements.append({'name': 'FloorNotPerimeter','FlagPerimeter': 1,'FlagSolarRadiation': 2,'direction': 14,
                       'AdjacentSpace':'Underfloor','FlagUnsteady': 1,'A':AEarthFloor,'L': None,'sunshade': None})
            
    #主たる居室、その他の居室、非居室の床面積比率
    rMain=d_common['MainOccupantRoomFloorArea']/d_common['TotalFloorArea']
    rOther=d_common['OtherOccupantRoomFloorArea']/d_common['TotalFloorArea']
    rNon=1-rMain-rOther
    
    elements=[]
    for d_element in d_elements:
        if d_element['direction']==15:
            elements.append({
                'name'                        : d_element['name'],
                'Room'                        : 'Underfloor',
                'FlagPerimeter'               : d_element['FlagPerimeter'],
                'FlagSolarRadiation'          : d_element['FlagSolarRadiation'],
                'direction'                   : d_element['direction'],
                'AdjacentSpace'               : d_element['AdjacentSpace'],
                'FlagUnsteady'                : d_element['FlagUnsteady'],
                'A'                           : d_element['A'],
                'L'                           : d_element['L'],
                'sunshade'                    : d_element['sunshade']
            })
        else:
            if d_element['A'] is not None:
                AMain=d_element['A']*rMain
                AOther=d_element['A']*rOther
                ANon=d_element['A']*rNon
            else:
                AMain=None
                AOther=None
                ANon=d_None       

            if d_element['L'] is not None:
                LMain=d_element['L']*rMain
                LOther=d_element['L']*rOther
                LNon=d_element['L']*rNon
            else:
                LMain=None
                LOther=None
                LNon=None 

            temporary=[{
                'name'                        : d_element['name'],
                'Room'                        : 'MainOccupantRoom',
                'FlagPerimeter'               : d_element['FlagPerimeter'],
                'FlagSolarRadiation'          : d_element['FlagSolarRadiation'],
                'direction'                   : d_element['direction'],
                'AdjacentSpace'               : d_element['AdjacentSpace'],
                'FlagUnsteady'                : d_element['FlagUnsteady'],
                'A'                           : AMain,
                'L'                           : LMain,
                'sunshade'                    : d_element['sunshade']
            },{
                'name'                        : d_element['name'],
                'Room'                        : 'OtherOccupantRoomFloorArea',
                'FlagPerimeter'               : d_element['FlagPerimeter'],
                'FlagSolarRadiation'          : d_element['FlagSolarRadiation'],
                'direction'                   : d_element['direction'],
                'AdjacentSpace'               : d_element['AdjacentSpace'],
                'FlagUnsteady'                : d_element['FlagUnsteady'],
                'A'                           : AOther,
                'L'                           : LOther,
                'sunshade'                    : d_element['sunshade']
            },{
                'name'                        : d_element['name'],
                'Room'                        : 'NonOccupantRoomFloorArea',
                'FlagPerimeter'               : d_element['FlagPerimeter'],
                'FlagSolarRadiation'          : d_element['FlagSolarRadiation'],
                'direction'                   : d_element['direction'],
                'AdjacentSpace'               : d_element['AdjacentSpace'],
                'FlagUnsteady'                : d_element['FlagUnsteady'],
                'A'                           : ANon,
                'L'                           : LNon,
                'sunshade'                    : d_element['sunshade']
            }]
            for i in range(0,3):
                elements.append(temporary[i])

    return elements

### Example

In [10]:
convert_element({
    'Region' : 6,
    'FlagSimplifiedEnvelopeInput': 1,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0
    },[{
            'name'                    : 'Floor',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 2,
            'direction'               : 12,
            'FlagUnsteady'            : 1,
            'A'                       : 30,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'EarthFloor',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 2,
            'direction'               : 15,
            'FlagUnsteady'            : 1,
            'A'                       : 30,
            'L'                       : None,
            'sunshade'                : None
        }
    ]
)

[{'A': 7.5,
  'AdjacentSpace': None,
  'FlagPerimeter': 1,
  'FlagSolarRadiation': 2,
  'FlagUnsteady': 1,
  'L': None,
  'Room': 'MainOccupantRoom',
  'direction': 12,
  'name': 'Floor',
  'sunshade': None},
 {'A': 15.0,
  'AdjacentSpace': None,
  'FlagPerimeter': 1,
  'FlagSolarRadiation': 2,
  'FlagUnsteady': 1,
  'L': None,
  'Room': 'OtherOccupantRoomFloorArea',
  'direction': 12,
  'name': 'Floor',
  'sunshade': None},
 {'A': 7.5,
  'AdjacentSpace': None,
  'FlagPerimeter': 1,
  'FlagSolarRadiation': 2,
  'FlagUnsteady': 1,
  'L': None,
  'Room': 'NonOccupantRoomFloorArea',
  'direction': 12,
  'name': 'Floor',
  'sunshade': None},
 {'A': 30,
  'AdjacentSpace': None,
  'FlagPerimeter': 1,
  'FlagSolarRadiation': 2,
  'FlagUnsteady': 1,
  'L': None,
  'Room': 'Underfloor',
  'direction': 15,
  'name': 'EarthFloor',
  'sunshade': None},
 {'A': 7.5,
  'AdjacentSpace': 'Underfloor',
  'FlagPerimeter': 1,
  'FlagSolarRadiation': 2,
  'FlagUnsteady': 1,
  'L': None,
  'Room': 'MainOccu

## 2-6. コンバート【全分類】

In [11]:
def Convert(d):
    d_common=convert_common(d['Common'])
    d_ceiling=convert_wall(d['Ceiling'])
    d_wall=convert_wall(d['Wall'])
    d_floor=convert_wall(d['Floor'])
    d_window=convert_window(d['Window'])
    d_door=convert_window(d['Door'])
    d_sunshade=convert_sunshade(d['Sunshade'])

    temporary=convert_element(d_common, d['Elements'])
    d_element=[]
    for e in temporary:
        if e['name']=='Ceiling':
            e['Wall']=d_ceiling
        elif e['name']=='Wall':
            e['Wall']=d_wall
        elif e['name']=='Floor':
            e['Wall']=d_floor
        elif e['name']=='EarthFloor':
            e['FloorNonPerimeter']={
                    'name'                       : 'FloorNotPerimeter',
                    'structure'                  : 1,
                    'type'                       : 3,
                    'Layers'                     : [{'material': 'PED','d': 0.012,'rambda': 0.16,'Crho': 720}],
                    'Ua'                         : 3.77,
                    'Ug'                         : None,
                    'HeatBridge'                 : None,
                    'rHeatBridge'                : None,
                    'UHeatBridge'                : None,
                    'lHeatBridge'                : None,
                    'psiHeatBridge'              : None,
                    'Ur'                         : None
                }
        elif e['name']=='Window':
            e['Window']=d_window
            e['Sunshade']=d_sunshade
        elif e['name']=='Door':
            e['Door']=d_door
        d_element.append(e)

    return {'common':d_common,'element':d_element}

In [12]:
d = {
    'Common' : {
        'Region'                     : 6,
        'FlagSimplifiedEnvelopeInput': 2,
        'MainOccupantRoomFloorArea'  : 30.0,
        'OtherOccupantRoomFloorArea' : 60.0,
        'TotalFloorArea'             : 120.0
    },
    'Ceiling' : {
        'name'                       : 'Ceiling',
        'structure'                  : 1,
        'type'                       : 1,
        'Layers'                     : [{'material': 'GW16K','d': 0.179,'rambda': 0.045,'Crho': 13},
                                        {'material': 'GPB','d': 0.0095,'rambda': 0.22,'Crho': 830}],
        'Ua'                         : 0.24,
        'Ug'                         : None,
        'HeatBridge'                 : None,
        'rHeatBridge'                : None,
        'UHeatBridge'                : None,
        'lHeatBridge'                : None,
        'psiHeatBridge'              : None,
        'Ur'                         : None
    },
    'Wall' : {
        'name'                       : 'Wall',
        'structure'                  : 1,
        'type'                       : 2,
        'Layers'                     : [{'material': 'GW16K','d': 0.076,'rambda': 0.045,'Crho': 13},
                                        {'material': 'GPB','d': 0.0095,'rambda': 0.22,'Crho': 830}],
        'Ua'                         : 0.53,
        'Ug'                         : None,
        'HeatBridge'                 : None,
        'rHeatBridge'                : None,
        'UHeatBridge'                : None,
        'lHeatBridge'                : None,
        'psiHeatBridge'              : None,
        'Ur'                         : None
    },
    'Floor' : {
        'name'                       : 'Floor',
        'structure'                  : 1,
        'type'                       : 3,
        'Layers'                     : [{'material': 'GW16K','d': 0.081,'rambda': 0.045,'Crho': 13},
                                        {'material': 'PED','d': 0.012,'rambda': 0.16,'Crho': 720}],
        'Ua'                         : 0.48,
        'Ug'                         : None,
        'HeatBridge'                 : None,
        'rHeatBridge'                : None,
        'UHeatBridge'                : None,
        'lHeatBridge'                : None,
        'psiHeatBridge'              : None,
        'Ur'                         : None
    },
    'Window' : {
        'name'                        : 'Window',
        'eta'                         : 0.738,
        'etaCooling'                  : None,
        'etaHeating'                  : None,
        'U'                           : 4.65
    },
    'Door' : {
        'name'                        : 'Door',
        'eta'                         : round(2.33*0.034,3),
        'etaCooling'                  : None,
        'etaHeating'                  : None,
        'U'                           : 2.33
    },
    'Sunshade' : {
        'name'                        : 'Sunshade',
        'y1'                          : 0.5,
        'y2'                          : 2, 
        'z'                           : 1,
        'fCooling'                    : None,
        'fHeating'                    : None
    },
    'Elements' : [{
            'name'                    : 'Ceiling',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 1,
            'FlagUnsteady'            : 1,
            'A'                       : 60,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'Wall',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 6,
            'FlagUnsteady'            : 1,
            'A'                       : 60,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'Floor',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 2,
            'direction'               : 12,
            'FlagUnsteady'            : 1,
            'A'                       : 30,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'EarthFloor',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 2,
            'direction'               : 15,
            'FlagUnsteady'            : 1,
            'A'                       : 30,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'Window',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 6,
            'FlagUnsteady'            : 2,
            'A'                       : 6,
            'L'                       : None,
            'sunshade'                : 'Sunshade'
        },{
            'name'                    : 'Door',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 2,
            'FlagUnsteady'            : 2,
            'A'                       : 3,
            'L'                       : None,
            'sunshade'                : None
        }
    ]
}

Convert(d)

{'common': {'FlagSimplifiedEnvelopeInput': 2,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'element': [{'A': 15.0,
   'AdjacentSpace': None,
   'FlagPerimeter': 1,
   'FlagSolarRadiation': 1,
   'FlagUnsteady': 1,
   'L': None,
   'Room': 'MainOccupantRoom',
   'Wall': {'HeatBridge': None,
    'Layers': [{'Crho': 13, 'd': 0.179, 'material': 'GW16K', 'rambda': 0.045},
     {'Crho': 830, 'd': 0.0095, 'material': 'GPB', 'rambda': 0.22}],
    'UHeatBridge': None,
    'Ua': 0.24,
    'Ug': None,
    'Ur': None,
    'lHeatBridge': None,
    'name': 'Ceiling',
    'psiHeatBridge': None,
    'rHeatBridge': None,
    'structure': 1,
    'type': 1},
   'direction': 1,
   'name': 'Ceiling',
   'sunshade': None},
  {'A': 30.0,
   'AdjacentSpace': None,
   'FlagPerimeter': 1,
   'FlagSolarRadiation': 1,
   'FlagUnsteady': 1,
   'L': None,
   'Room': 'OtherOccupantRoomFloorArea',
   'Wall': {'HeatBridge': None,
    'Layers': [